In [ ]:
import json
import re
import os
import sys

import requests

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# First we need to download the root.zone from IANA

res = requests.get("https://www.internic.net/domain/root.zone")
print(f"Status: {res.status_code}")

# Parse the zone file into tuples
root_zone = [l.split() for l in res.text.splitlines()]

# Then we filter on ccTLDs
# RegEx filter on ccTLD format in zone file
pattern = re.compile(r"^[a-z]{2}\.$")
ns_records = filter(lambda r: pattern.match(r[0]), root_zone)
# Lastly, filter on type='NS'
ns_records = filter(lambda r: r[3] == "NS", ns_records)

ns_records = list(ns_records)

df_ns = pd.DataFrame(data=ns_records, columns=["name", "ttl", "class", "type", "record"], dtype=str)
df_ns

We managed to get a currated list of all records within the root zone file which gives us the list of authoritative name servers for each ccTLD. Let's fetch the `A` and `AAAA` records accordingly.

In [ ]:
fqdns = [r[4] for r in ns_records]
a_records = list(filter(lambda r: r[0] in fqdns and r[3] == 'A', root_zone))
aaaa_records = list(filter(lambda r: r[0] in fqdns and r[3] == 'AAAA', root_zone))

df_ip = pd.concat([
    pd.DataFrame(data=a_records, columns=["fqdn", "ttl", "class", "type", "ip"]),
    pd.DataFrame(data=aaaa_records, columns=["fqdn", "ttl", "class", "type", "ip"]),
])

df_ip = df_ip[["fqdn", "ip"]]
df_ip

In [ ]:
ipv4 = sorted(list(set([r[4] for r in a_records])))
ipv6 = sorted(list(set([r[4] for r in aaaa_records])))

print(f"#IPv4: {len(ipv4)}")
print(f"#IPv6: {len(ipv6)}")

with open("/tmp/cctld_a.txt", "w") as f:
    f.write("\n".join(ipv4))

with open("/tmp/cctld_aaaa.txt", "w") as f:
    f.write("\n".join(ipv6))
    

We have a unique and sorted set of IPv4s and IPv6s for each of the authoritative domain name servers of the ccTLDs. We can fetch the IP information from `ipinfo.io` with a script I've used in the past to resolve a set of IPs.

Just run 

```
cat /tmp/cctld_a.txt | ipinfo.sh > /tmp/ipinfo_cctld_a.jsonl
cat /tmp/cctld_aaaa.txt | ipinfo.sh > /tmp/ipinfo_cctld_aaaa.jsonl
```

Afterwards we can load the JSON responses and evaluate how many IPs are anycasted.

In [ ]:
df = df_ns.merge(df_ip, how="inner", left_on="record", right_on="fqdn")
df = df[["name", "fqdn", "ip"]]
df

df_ipinfo = pd.concat([
    pd.read_json("../data/ipinfo_cctld_a.jsonl", lines=True),
    pd.read_json("../data/ipinfo_cctld_aaaa.jsonl", lines=True),
])

df = df.merge(df_ipinfo, how="left", left_on="ip", right_on="ip")
# Then do some data preparation
df["anycast"] = df["anycast"] == 1.0
df["ipv6"] = df["ip"].str.contains(":")
df_backup = df.copy()
# df[df["name"] == "ax."]
df

In [ ]:
df = df_backup.copy()
df = df[df.ipv6 == False]
df = df[["name", "fqdn", "ip", "anycast"]].reset_index(drop=True)
# df[["name", "anycast"]].drop_duplicates()
df.to_csv("../data/dns-ns-anycast.csv", index=False)

df = df[["name", "anycast"]].groupby("name").max().reset_index()

data = df.groupby("anycast").count().to_dict()["name"]
names = df[df["anycast"] == False]["name"].unique()

print(f"ccTLDs with at least one anycasted authoritative name server: {data[True]}")
print(f"ccTLDs with no anycasted authoritative name server: {data[False]}")

print("ccTLDs without any anycasted authoritative name server:")
names


We see that quite a number of TLDs do not have a single anycasted authoritative name server.
If we carefully inspect the list, we are surprised to find these ccTLDs in there:

```
by. - Belarus
eg. - Egypt
kr. - South Korea
md. - Moldova
ru. - Russia
su. - Soviet Union
uz. - Uzbekistan
ye. - Yemen
```

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

ax.pie(
    np.array([206, 41]),
    labels=["anycast\n(206 ccTLDs)", "unicast only\n(41 ccTLDs)"],
    startangle=90,
    explode=[0.0, 0.1],
    colors=["#eecc66", "#6699cc"],
    rotatelabels=False,
    autopct='~%1.0f%%',
)

ax.set_title("ccTLDs with Authoritative Name Servers announced under Anycast")
ax.set_facecolor('#ffffff')
fig.savefig("../img/cctld-anycast-ipv4.jpg", dpi=120, )